# Code Intepreter with Access To Other AWS Resources

**Code interpreter in Public environment is required to access any AWS resources other than S3.** 

We can access other AWS resource in the following ways
- Through command execution and aws CLI
- Through code execution and AWS SDK


Using CloudWatch Logs as an example.



In [1]:
region="us-east-1"

### Create Code Interpreter Execution Role

In [2]:
import boto3
import json

iam = boto3.client('iam')
role_name = "CodeInterpreterCloudWatchExecutionRole"
response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
            }
        ]
    }),
)
role_arn = response["Role"]["Arn"]

### Add Necessary Permissions

Using CloudWatch Logs as example here.

In [ ]:
policy_name = "log_access"
iam.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "logs:*"
                ],
                "Resource": [
                    "*"
                ]
            }
        ]
    })
)

{'ResponseMetadata': {'RequestId': '806e8e89-93d6-4c40-b592-dd373a38e201',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 03 Aug 2025 02:00:41 GMT',
   'x-amzn-requestid': '806e8e89-93d6-4c40-b592-dd373a38e201',
   'content-type': 'text/xml',
   'content-length': '206'},
  'RetryAttempts': 0}}

### Create Code Interpreter With Execution Role Above

In [6]:
control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region,
    endpoint_url=f"https://bedrock-agentcore-control.{region}.amazonaws.com"
)


In [7]:
response = control_client.create_code_interpreter(
    name="code_interpreter_cloud_watch",
    executionRoleArn=role_arn,
    networkConfiguration={
        "networkMode": "PUBLIC" # required to access any aws resources other than s3
    }
)
code_interpreter_id = response["codeInterpreterId"]
print(f"code interpreter created: {code_interpreter_id}")

code interpreter created: code_interpreter_cloud_watch-B4pozhd3xA


### Access AWS Resource

In [8]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
code_client = CodeInterpreter('us-east-1')
log_group_name_prefix = "/aws/bedrock-agentcore"

session_id = code_client.start(
    identifier=code_interpreter_id
)

#### With CLI Command

--no-cli-pager option is required. Otherwise, we will get *Unable to redirect output to pager* error.

In [9]:
response = code_client.invoke("executeCommand", {
    "command": f"aws logs describe-log-groups --region {region} --max-item 1 --no-cli-pager --log-group-name-prefix {log_group_name_prefix}",
})
for event in response["stream"]:
    result = event["result"]
    for content in result["content"]:
        print(content["text"])


{
    "logGroups": [
        {
            "logGroupName": "/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT",
            "creationTime": 1753776049424,
            "metricFilterCount": 0,
            "arn": "arn:aws:logs:us-east-1:075198889659:log-group:/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT:*",
            "storedBytes": 1182041,
            "logGroupClass": "STANDARD",
            "logGroupArn": "arn:aws:logs:us-east-1:075198889659:log-group:/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT"
        }
    ],
    "NextToken": "eyJuZXh0VG9rZW4iOiBudWxsLCAiYm90b190cnVuY2F0ZV9hbW91bnQiOiAxfQ=="
}



#### With Boto3

##### Install

boto3 is not one of the libraries included

In [10]:
response = code_client.invoke("executeCommand", {
    "command": "pip install boto3",
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))


{
  "content": [
    {
      "type": "text",
      "text": "Collecting boto3\r\n  Downloading boto3-1.40.1-py3-none-any.whl.metadata (6.7 kB)\r\nCollecting botocore<1.41.0,>=1.40.1 (from boto3)\r\n  Downloading botocore-1.40.1-py3-none-any.whl.metadata (5.7 kB)\r\nCollecting jmespath<2.0.0,>=0.7.1 (from boto3)\r\n  Downloading jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)\r\nCollecting s3transfer<0.14.0,>=0.13.0 (from boto3)\r\n  Downloading s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)\r\nRequirement already satisfied: python-dateutil<3.0.0,>=2.1 in /opt/amazon/genesis1p-tools/venv/lib/python3.10/site-packages (from botocore<1.41.0,>=1.40.1->boto3) (2.9.0.post0)\r\nRequirement already satisfied: urllib3!=2.2.0,<3,>=1.25.4 in /opt/amazon/genesis1p-tools/python3.10/lib/python3.10/site-packages (from botocore<1.41.0,>=1.40.1->boto3) (1.26.19)\r\nRequirement already satisfied: six>=1.5 in /opt/amazon/genesis1p-tools/venv/lib/python3.10/site-packages (from python-dateutil<3.0.0,

##### Execute Code

In [11]:
response = code_client.invoke("executeCode", {
    "language": "python",
    "code": f"""
import boto3
import json
logs = boto3.client("logs", region_name='{region}')
response = logs.describe_log_groups(
    logGroupNamePrefix="{log_group_name_prefix}",
    limit=1
)
for log in response["logGroups"]:
    print(json.dumps(log, indent=2))
    """
})
for event in response["stream"]:
    result = event["result"]
    for content in result["content"]:
        print(content["text"])

{
  "logGroupName": "/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT",
  "creationTime": 1753776049424,
  "metricFilterCount": 0,
  "arn": "arn:aws:logs:us-east-1:075198889659:log-group:/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT:*",
  "storedBytes": 1182041,
  "logGroupClass": "STANDARD",
  "logGroupArn": "arn:aws:logs:us-east-1:075198889659:log-group:/aws/bedrock-agentcore/runtimes/runtime_demo-Tf8GmRCAKe-DEFAULT"
}


### Clean up

In [ ]:

agent_core = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    endpoint_url=f"https://bedrock-agentcore.{region}.amazonaws.com"
)
response = agent_core.list_code_interpreter_sessions(
    codeInterpreterIdentifier=code_interpreter_id,
    status="READY" # 'READY'|'TERMINATED'
)

# stop all the running sessions
for item in response["items"]:
    agent_core.stop_code_interpreter_session(
        codeInterpreterIdentifier=code_interpreter_id,
        sessionId=item["sessionId"]
    )

response = control_client.delete_code_interpreter(
    codeInterpreterId = code_interpreter_id
)

# delete role policy and role
iam.delete_role_policy(
    RoleName=role_name,
    PolicyName=policy_name
)

iam.delete_role(
    RoleName = role_name
)
print("Deleted!")

Deleted!
